In [ ]:
%load_ext autoreload
%autoreload 2


## Libraries

In [ ]:
import pathlib
from pathlib import Path
import ee
import geemap


In [ ]:
# Add root to path
import sys

sys.path.append("..")
from component.script.gee.ee_fao_gaul import (
    get_fao_gaul_features,
    get_fao_gaul_subj,
)
from component.script.gee.dask_ee_vector_export import export_vector_with_dask
from component.script.gee.dask_ee_raster_export import export_raster_with_dask
from component.script.gee.ee_rasterize_unique_values import gee_rasterize_unique_values


## GEE


In [ ]:
ee_project = "ee-joseserafini-fao"
ee.Authenticate()
ee.Initialize(project=ee_project)


## Dask Download Instance

In [ ]:
from dask.distributed import LocalCluster, Client

cluster = LocalCluster(n_workers=1, threads_per_worker=1, memory_limit="2GB")
download_client = Client(cluster)
download_client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40809,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:34051,Total threads: 1
Dashboard: http://127.0.0.1:38671/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:41695,


## Create folders

In [ ]:
from pathlib import Path

root_folder: Path = Path.cwd().parent
downloads_folder: Path = root_folder / "data"
downloads_folder.mkdir(parents=True, exist_ok=True)


## Set user parameters

In [ ]:
project_name = "test"
iso_code = "MTQ"
years: list[int] = [2015, 2020, 2024]
forest_source = "gfc"
tree_cover_threshold = 10


## Create projects folders

In [ ]:
project_folder = downloads_folder / project_name
project_folder.mkdir(parents=True, exist_ok=True)

data_raw_folder = project_folder / "data_raw"
data_raw_folder.mkdir(parents=True, exist_ok=True)


## Helpers

In [ ]:
from pathlib import Path


def set_variable_file_path(
    project_name: str, data_raw_folder: str, variable_name: str, variable_type: str
) -> Path:
    """
    Determine the file extension based on variable type and return the full file path.

    Args:
        project_name (str): The name of the project.
        data_raw_folder (str): The path to the data raw folder.
        variable_type (str): The type of the variable ("vector" or "raster").
        data_raw_folder (str): The path to the data raw folder.
        variable_name (str): The name of the variable.

    Returns:
        Path: The full file path with the appropriate extension.

    Raises:
        ValueError: If an unsupported variable type is provided.
    """
    # Define supported extensions
    extensions = {"vector": ".shp", "raster": ".tif"}

    # Validate variable type and get the corresponding extension
    if variable_type not in extensions:
        raise ValueError(
            f"Unsupported variable type: {variable_type}. Supported types are: {list(extensions.keys())}"
        )

    variable_extension = extensions[variable_type]

    # Create and return the full file path
    return Path(data_raw_folder) / f"{project_name}_{variable_name}{variable_extension}"


In [ ]:
from pathlib import Path
import shutil
from typing import Union


def copy_and_rename_file(
    file_path: Union[str, Path], destination_path: Union[str, Path]
) -> Path:
    """
    Copy a file to a designated location and rename it. If the source is a shapefile (.shp),
    also copies all corresponding auxiliary files.

    Args:
        file_path (Union[str, Path]): The path to the source file to be copied.
        destination_path (Union[str, Path]): The full path including folder and new filename for the copied file.

    Returns:
        Path: The path to the newly copied file.

    Raises:
        FileNotFoundError: If the source file does not exist.
        PermissionError: If there are insufficient permissions to read the source file or write to destination.

    Example:
        >>> copy_and_rename_file('/path/to/source/file.shp', '/path/to/destination/renamed_file.shp')
        PosixPath('/path/to/destination/renamed_file.shp')
    """
    # Convert to Path objects
    source_file = Path(file_path)
    new_file_path = Path(destination_path)

    # Validate that source file exists
    if not source_file.exists():
        raise FileNotFoundError(f"Source file '{source_file}' does not exist.")

    # Ensure the destination directory exists
    new_file_path.parent.mkdir(parents=True, exist_ok=True)

    # Copy the main file to the destination with a new name
    shutil.copy2(source_file, new_file_path)

    # If source is a shapefile (.shp), also copy auxiliary files
    if source_file.suffix.lower() == ".shp":
        # Use pathlib to construct the glob pattern and iterate over auxiliary files
        aux_files = list(source_file.parent.glob(f"{source_file.stem}.*"))
        # print(f"Found {len(aux_files)} auxiliary files: {aux_files}")
        for aux_file in aux_files:
            new_aux_filename = f"{new_file_path.stem}{aux_file.suffix}"
            new_aux_path = new_file_path.parent / new_aux_filename
            shutil.copy2(aux_file, new_aux_path)
            print(f"Auxiliary file copied to {new_aux_path}")

    # print(f"File copied to {new_file_path}")
    return new_file_path


## AOI

In [ ]:
aoi_source = "local"
aoi_type = "vector"
aoi_name: Path = set_variable_file_path(project_name, data_raw_folder, "aoi", aoi_type)


In [ ]:
if aoi_source == "gee":
    ##Define AOI using a GEE featureCollecion
    # aoi_id = "str"
    # aoi = ee.FeatureCollection(aoi_id)

    ##Define AOI using a FAO Gaul Object
    aoi = get_fao_gaul_features(level=0, code=iso_code)

    if not Path(aoi_name).exists():
        export_vector_with_dask(
            aoi, aoi_name, selectors=["gaul0_name", "iso3_code"], download_client, project=ee_project
        )


In [ ]:
if aoi_source == "local":
    aoi_path: Path = (
        "/home/jose/workspace/deforisk-jupyter-nb-v2/data/MTQ/other/NAD_sabanasYari.shp"
    )
    copy_and_rename_file(aoi_path, aoi_name)

    aoi = geemap.shp_to_ee(aoi_path)


In [ ]:
aoi_geom = aoi.geometry()
aoi_geometry_json = aoi_geom.serialize()


## SubJuridistion

In [ ]:
subj_source = "gee"
subj_type = "raster"


In [ ]:
subj_name: Path = set_variable_file_path(
    project_name, data_raw_folder, "subj", subj_type
)
subj_name


In [ ]:
if subj_source == "gee":
    filtered_subj, filtered_attribute = get_fao_gaul_subj(2, aoi)
    subj_image = (
        ee.Image(gee_rasterize_unique_values(filtered_subj, "gaul2_name"))
        .clip(aoi)
        .toByte()
    )
    if not Path(subj_name).exists():
        export_raster_with_dask(
            subj_image,
            subj_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if subj_source == "local":
    subj_path: Path = ""
    copy_and_rename_file(subj_path, subj_name)


## Protected Areas

In [ ]:
pa_source = "gee"
pa_type = "raster"


In [ ]:
pa_name: Path = set_variable_file_path(project_name, data_raw_folder, "pa", subj_type)


In [ ]:
if subj_source == "gee":
    # Source
    # https://developers.google.com/earth-engine/datasets/catalog/WCMC_WDPA_current_polygons

    wdpa_poly = (
        ee.FeatureCollection("WCMC/WDPA/current/polygons")
        .filterBounds(aoi)
        .filter(
            ee.Filter.inList(
                "STATUS", ["Designated", "Inscribed", "Established", "Proposed"]
            )
        )
    )
    wdpa_image = (
        wdpa_poly.reduceToImage(["WDPAID"], ee.Reducer.first())
        .gt(0)
        .unmask()
        .clip(aoi)
        .toByte()
    )
    if not Path(pa_name).exists():
        export_raster_with_dask(
            wdpa_image,
            pa_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if pa_source == "local":
    pa_path: Path = ""
    copy_and_rename_file(pa_path, pa_name)


## Altitude and Slope

In [ ]:
altitude_source = "gee"
altitude_type = "raster"
slope_source = "gee"
slope_type = "raster"


In [ ]:
altitude_name: Path = set_variable_file_path(
    project_name, data_raw_folder, "altitude", altitude_type
)
slope_name: Path = set_variable_file_path(
    project_name, data_raw_folder, "slope", slope_type
)


In [ ]:
if altitude_source == "gee":
    srtm = ee.Image("USGS/SRTMGL1_003").select("elevation").clip(aoi)
    if not Path(altitude_name).exists():
        export_raster_with_dask(
            srtm,
            altitude_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            nodata_value=None,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if slope_source == "gee":
    slope = ee.Terrain.slope(srtm).clip(aoi)
    if not Path(slope_name).exists():
        export_raster_with_dask(
            slope,
            slope_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            nodata_value=None,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if altitude_source == "local":
    altitude_path: Path = ""
    copy_and_rename_file(altitude_path, altitude_name)


In [ ]:
if slope_source == "local":
    slope_path: Path = ""
    copy_and_rename_file(slope_path, slope_name)


## Forest layers

In [ ]:
forest_source_primary = "gee"
forest_source_secondary = forest_source
forest_type = "raster"


In [ ]:
if forest_source_primary == "gee":
    if forest_source_secondary == "gfc":
        forest_var = (
            "forest_" + forest_source_secondary + "_" + str(tree_cover_threshold) + "_"
        )
    elif forest_source_secondary == "tmf":
        forest_var = "forest_" + forest_source_secondary + "_"
elif forest_source_primary == "local":
    forest_var = "forest_"


In [ ]:
forest1_name: Path = set_variable_file_path(
    project_name, data_raw_folder, forest_var + str(years[0]), forest_type
)
forest2_name: Path = set_variable_file_path(
    project_name, data_raw_folder, forest_var + str(years[1]), forest_type
)
forest3_name: Path = set_variable_file_path(
    project_name, data_raw_folder, forest_var + str(years[2]), forest_type
)


In [ ]:
if forest_source_primary == "gee" and forest_source == "gfc":
    gfcImage = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")
    clipGfc = gfcImage.clip(aoi)
    forest2000 = clipGfc.select(["treecover2000"])
    forest2000_thr = (
        ee.Image(0).where(forest2000.gte(tree_cover_threshold), 1).clip(aoi)
    )
    loss = clipGfc.select(["lossyear"])

    forest_gcf_t1 = forest2000_thr.where(loss.lt(years[0] - 2000), 0).rename("B1")
    forest_gcf_t2 = forest2000_thr.where(loss.lt(years[1] - 2000), 0).rename("B1")
    forest_gcf_t3 = forest2000_thr.where(loss.lt(years[2] - 2000), 0).rename("B1")

    if not Path(forest1_name).exists():
        export_raster_with_dask(
            forest_gcf_t1,
            forest1_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )

    if not Path(forest2_name).exists():
        export_raster_with_dask(
            forest_gcf_t2,
            forest2_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )
    if not Path(forest3_name).exists():
        export_raster_with_dask(
            forest_gcf_t3,
            forest3_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if forest_source_primary == "gee" and forest_source == "tmf":
    tmfImage = (
        ee.ImageCollection("projects/JRC/TMF/v1_2024/AnnualChanges")
        .filterBounds(aoi)
        .mosaic()
    )
    forest2_t1 = tmfImage.select("Dec" + str(years[0] - 1))
    forest2_t2 = tmfImage.select("Dec" + str(years[1] - 1))
    forest2_t3 = tmfImage.select("Dec" + str(years[2] - 1))

    forest_tmf_t1 = (
        forest2_t1.where(forest2_t1.eq(2), 1).where(forest2_t1.neq(1), 0).rename("B1")
    )
    forest_tmf_t2 = (
        forest2_t2.where(forest2_t2.eq(2), 1).where(forest2_t2.neq(1), 0).rename("B2")
    )
    forest_tmf_t3 = (
        forest2_t3.where(forest2_t3.eq(2), 1).where(forest2_t3.neq(1), 0).rename("B3")
    )
    if not Path(forest1_name).exists():
        export_raster_with_dask(
            forest_tmf_t1,
            forest1_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )

    if not Path(forest2_name).exists():
        export_raster_with_dask(
            forest_tmf_t2,
            forest2_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )
    if not Path(forest3_name).exists():
        export_raster_with_dask(
            forest_tmf_t3,
            forest3_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if forest_source_primary == "local":
    forest1_path: Path = ""
    copy_and_rename_file(forest1_path, forest1_name)
    forest2_path: Path = ""
    copy_and_rename_file(forest2_path, forest2_name)
    forest3_path: Path = ""
    copy_and_rename_file(forest3_path, forest3_name)


## Rivers

In [ ]:
rivers_source = "gee"
rivers_type = "raster"


In [ ]:
rivers_name: Path = set_variable_file_path(
    project_name, data_raw_folder, "rivers", rivers_type
)


In [ ]:
if rivers_source == "gee":
    # Source
    # https://gee-community-catalog.org/projects/osm_water/
    osm_water = (
        ee.ImageCollection("projects/sat-io/open-datasets/OSM_waterLayer")
        .filterBounds(aoi)
        .mosaic()
        .clip(aoi)
    )
    osm_rivers = osm_water.gte(2).unmask().clip(aoi).toByte()

    if not Path(rivers_name).exists():
        export_raster_with_dask(
            osm_rivers,
            rivers_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if rivers_source == "local":
    rivers_path: Path = ""
    copy_and_rename_file(rivers_path, rivers_name)


## Roads

In [ ]:
roads_source = "gee"
roads_type = "raster"


In [ ]:
roads_name: Path = set_variable_file_path(
    project_name, data_raw_folder, "roads", roads_type
)


In [ ]:
if roads_source == "gee":
    osm_roads = (
        ee.Image(
            "projects/ee-andyarnellgee/assets/crosscutting/infrastructure/roads_osm/roadsAllImageOSM"
        )
        .unmask()
        .clip(aoi)
        .toByte()
    )
    if not Path(roads_name).exists():
        export_raster_with_dask(
            osm_roads,
            roads_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if roads_source == "local":
    roads_path: Path = ""
    copy_and_rename_file(roads_path, roads_name)


## Towns

In [ ]:
towns_source = "gee"
towns_type = "raster"


In [ ]:
def closest_epoch(year):
    # Define the list of epochs from 1975 to 2020 with 5-year intervals
    epochs = list(range(1975, 2021, 5))

    # Find the closest epoch to the given year
    closest = min(epochs, key=lambda x: abs(x - year))

    return closest


# Apply the function to each year
closest_epochs = [closest_epoch(y) for y in years]


In [ ]:
if towns_source == "gee":
    towns1_name: Path = set_variable_file_path(
        project_name, data_raw_folder, "towns_" + str(closest_epochs[0]), roads_type
    )
    towns2_name: Path = set_variable_file_path(
        project_name, data_raw_folder, "towns_" + str(closest_epochs[1]), roads_type
    )
    towns3_name: Path = set_variable_file_path(
        project_name, data_raw_folder, "towns_" + str(closest_epochs[2]), roads_type
    )
elif towns_source == "local":
    towns_name: Path = set_variable_file_path(
        project_name, data_raw_folder, "towns", roads_type
    )


In [ ]:
if towns_source == "gee":
    jrc_ghsl_pop = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")

    cities_pop_1 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[0]))
    cities_pop_2 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[1]))
    cities_pop_3 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[2]))

    cities_build_1 = ee.Image(
        "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[0])
    ).select("built_surface")
    cities_build_2 = ee.Image(
        "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[1])
    ).select("built_surface")
    cities_build_3 = ee.Image(
        "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[2])
    ).select("built_surface")

    cities1 = (
        ee.Image(0).where(cities_pop_1.gte(15).And(cities_build_1.gte(90)), 1).clip(aoi)
    )
    cities2 = (
        ee.Image(0).where(cities_pop_2.gte(15).And(cities_build_2.gte(90)), 1).clip(aoi)
    )
    cities3 = (
        ee.Image(0).where(cities_pop_3.gte(15).And(cities_build_3.gte(90)), 1).clip(aoi)
    )
    if not Path(towns1_name).exists():
        export_raster_with_dask(
            cities_build_1,
            towns1_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )

    if not Path(towns2_name).exists():
        export_raster_with_dask(
            cities_build_2,
            towns2_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )
    if not Path(towns3_name).exists():
        export_raster_with_dask(
            cities_build_3,
            towns3_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if towns_source == "local":
    towns_path: Path = ""
    copy_and_rename_file(towns_path, towns_name)


In [ ]:
## Consider using Global Human Modification v3
# https://gee-community-catalog.org/projects/ghm/?h=human

## Oxford accessibility to cities 2015
## ee.Image('Oxford/MAP/accessibility_to_cities_2015_v1_0')


## Custom variable

In [ ]:
other_source = "gee"
other_type = "raster"
other_variable_name = "mineria"


In [ ]:
other_name: Path = set_variable_file_path(
    project_name, data_raw_folder, other_variable_name, other_type
)


In [ ]:
if other_source == "gee":
    other_gee_id = "str"
    other_image = ee.Image(aoi_id)

    if not Path(other_name).exists():
        export_raster_with_dask(
            other_image,
            other_name,
            scale=30,
            crs="EPSG:4326",
            region=aoi_geom,
            overwrite=True,
            unmask_value=255,
            nodata_value=255,
            client=download_client,
            project=ee_project,
        )


In [ ]:
if other_source == "local":
    others_path: Path = ""
    copy_and_rename_file(others_path, others_name)


## Save parameters


In [ ]:
# def save_selected_keys_to_txt(self, keys_to_save: list, filename: str):
#         """Saves selected keys of a dictionary to a text file in 'key= value' format.
#         If a key contains a dictionary, its contents are written as separate key-value pairs."""
#         nfile = self.args["workdir"] + '/' + filename
#         print(nfile)

#         with open(nfile, "w") as file:
#             for key in keys_to_save:
#                 if key in self.args:  # Only write if key exists in the dictionary
#                     value = self.args[key]
#                     if isinstance(value, dict):  # If value is a dictionary, write its contents
#                         for sub_key, sub_value in value.items():
#                             file.write(f"{sub_key}= {sub_value}\n")
#                     else:
#                         file.write(f"{key}= {value}\n")


In [ ]:
# save_selected_keys_to_txt(['get_fcc_args','isocode', 'proj'], 'parameters.txt')
